<a href="https://colab.research.google.com/github/m0rTI88/weather-dashboard-react/blob/main/Secure_ATM_Simulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json, os
from datetime import datetime, date, timedelta
import uuid
import hashlib

# ====== GLOBAL SETTINGS ======
CURRENCY = "UZS"          # основная валюта системы
TRANSFER_FEE_PERCENT = 1  # комиссия 1% на переводы между картами


# ===== helper for "SMS" =====
def send_sms_mock(phone, message):
    if phone:
        print(f"\n📲 SMS to {phone}: {message}\n")


# ========== BANK CLASS ==========
class Bank:
    def __init__(self, db_file="bank_data.json", index_file="user_index.json", log_file="security_log.json"):
        self.db_file = db_file
        self.index_file = index_file
        self.log_file = log_file
        self.data = self.load_data()
        self.user_index = self.load_index()
        self.logs = self.load_logs()

        # если старый файл без ключа bank_fee – добавим
        if "bank_fee" not in self.data:
            self.data["bank_fee"] = 0
            self.save_data()

    # -------- basic storage --------
    def load_data(self):
        if not os.path.exists(self.db_file):
            data = {
                "admins": {
                    "admin": "admin123"
                },
                "users": {},
                "atm_cash": {
                    "2000": 10,
                    "500": 20,
                    "200": 30,
                    "100": 40
                },
                "bank_fee": 0  # сюда накапливается комиссия
            }
            with open(self.db_file, "w") as f:
                json.dump(data, f, indent=4)
            return data
        with open(self.db_file, "r") as f:
            return json.load(f)

    def save_data(self):
        with open(self.db_file, "w") as f:
            json.dump(self.data, f, indent=4)

    def load_index(self):
        if not os.path.exists(self.index_file):
            return {}
        with open(self.index_file, "r") as f:
            return json.load(f)

    def save_index(self):
        with open(self.index_file, "w") as f:
            json.dump(self.user_index, f, indent=4)

    def load_logs(self):
        if not os.path.exists(self.log_file):
            return []
        with open(self.log_file, "r") as f:
            return json.load(f)

    def save_logs(self):
        with open(self.log_file, "w") as f:
            json.dump(self.logs, f, indent=4)

    # -------- id / log helpers --------
    @staticmethod
    def generate_user_id():
        raw = uuid.uuid4().hex
        return hashlib.sha256(raw.encode()).hexdigest()

    def log_action(self, user, action, status, details=None):
        entry = {
            "datetime": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "user_id": user.get("user_id") if user else None,
            "name": user.get("name") if user else None,
            "action": action,
            "status": status,
            "details": details or {}
        }
        self.logs.append(entry)
        self.save_logs()

    def get_logs_by_user_id(self, user_id, limit=None):
        """Возвращает логи по конкретному user_id (hash)."""
        result = [log for log in self.logs if log.get("user_id") == user_id]
        if limit:
            result = result[-limit:]
        return result

    def get_logs_by_card(self, card, limit=None):
        """Ищем пользователя по номеру карты и показываем его логи (через user_id)."""
        user = self.get_user(card)
        if not user:
            return []
        user_id = user.get("user_id")
        return self.get_logs_by_user_id(user_id, limit)

    # ---- Admin ----
    def validate_admin(self, username, password):
        return self.data["admins"].get(username) == password

    def add_user(self, card, pin, name, phone):
        if card in self.data["users"]:
            return False, "Card already exists"
        user_id = self.generate_user_id()
        user_obj = {
            "user_id": user_id,
            "login": card,         # логин считаем номером карты
            "name": name,
            "phone": phone,
            "pin": str(pin),
            "status": "active",
            "failed_attempts": 0,
            "accounts": {
                "savings": 0,
                "current": 0
            },
            "transactions": [],
            "withdrawn_today": 0,
            "last_withdraw_date": None
        }
        self.data["users"][card] = user_obj
        # отдельный индекс по user_id
        self.user_index[user_id] = {
            "card": card,
            "login": card,
            "name": name,
            "phone": phone
        }
        self.save_data()
        self.save_index()
        self.log_action(user_obj, "create_user", "ok", {"card": card})
        return True, f"User added, internal id: {user_id}"

    def get_user(self, card):
        return self.data["users"].get(card)

    def get_user_by_id(self, user_id):
        info = self.user_index.get(user_id)
        if not info:
            return None
        card = info["card"]
        return self.get_user(card)

    def lock_card(self, card):
        u = self.get_user(card)
        if not u:
            return False
        u["status"] = "locked"
        self.save_data()
        self.log_action(u, "lock_card", "ok", {"card": card})
        return True

    def unlock_card(self, card):
        u = self.get_user(card)
        if not u:
            return False
        u["status"] = "active"
        u["failed_attempts"] = 0
        self.save_data()
        self.log_action(u, "unlock_card", "ok", {"card": card})
        return True

    # ---- PIN / login ----
    def check_pin(self, card, pin):
        user = self.get_user(card)
        if not user:
            return False, "Card not found"
        if user["status"] == "locked":
            return False, "Card is locked"
        if user["pin"] == str(pin):
            user["failed_attempts"] = 0
            self.save_data()
            return True, "OK"
        user["failed_attempts"] += 1
        if user["failed_attempts"] >= 3:
            user["status"] = "locked"
            self.save_data()
            self.log_action(user, "login", "locked", {"reason": "too many attempts"})
            return False, "Card locked due to too many wrong attempts"
        self.save_data()
        self.log_action(user, "login", "wrong_pin", {})
        return False, "Wrong PIN"

    # ---- Accounts ----
    def get_balance(self, card, account):
        user = self.get_user(card)
        if not user:
            return None
        return user["accounts"].get(account)

    def deposit(self, card, account, amount):
        user = self.get_user(card)
        if not user or account not in user["accounts"] or amount <= 0:
            if user:
                self.log_action(
                    user,
                    "deposit",
                    "failed",
                    {"account": account, "amount": amount, "currency": CURRENCY}
                )
            return False
        if user["status"] != "active":
            self.log_action(
                user,
                "deposit",
                "failed",
                {"reason": "locked", "amount": amount, "currency": CURRENCY}
            )
            return False
        user["accounts"][account] += amount
        self._add_tx(user, account, "Deposit", amount)
        self.save_data()
        self.log_action(
            user,
            "deposit",
            "ok",
            {"account": account, "amount": amount, "currency": CURRENCY}
        )
        return True

    # ---- ATM cash / withdraw ----
    def calculate_notes(self, amount):
        atm = self.data["atm_cash"]
        notes_needed = {}
        remaining = amount
        for note in [2000, 500, 200, 100]:
            count = min(remaining // note, atm.get(str(note), 0))
            if count > 0:
                notes_needed[note] = count
                remaining -= note * count
        return notes_needed if remaining == 0 else None

    def _reset_withdrawn_if_new_day(self, user):
        today = date.today().isoformat()
        if user["last_withdraw_date"] != today:
            user["last_withdraw_date"] = today
            user["withdrawn_today"] = 0

    def withdraw(self, card, account, amount, daily_limit=5000):
        user = self.get_user(card)
        if not user:
            return False, "Card not found"
        if user["status"] != "active":
            return False, "Card is locked"
        if account not in user["accounts"]:
            return False, "No such account"
        if amount <= 0:
            return False, "Amount must be positive"
        if user["accounts"][account] < amount:
            return False, "Insufficient funds"

        self._reset_withdrawn_if_new_day(user)
        if user["withdrawn_today"] + amount > daily_limit:
            return False, f"Daily limit {daily_limit} {CURRENCY} exceeded"

        notes = self.calculate_notes(amount)
        if notes is None:
            return False, "ATM cannot dispense this amount with available notes"

        user["accounts"][account] -= amount
        user["withdrawn_today"] += amount
        for note, cnt in notes.items():
            self.data["atm_cash"][str(note)] -= cnt

        self._add_tx(user, account, "Withdrawal", amount, extra={"notes": notes})
        self.save_data()
        self.log_action(
            user,
            "withdraw",
            "ok",
            {"account": account, "amount": amount, "currency": CURRENCY, "notes": notes}
        )
        return True, notes

    # ---- Transfer with 1% FEE + учёт комиссии банку ----
    def transfer(self, from_card, to_card, account, amount):
        """
        Перевод между картами:
        - списывается amount + fee (1%) с отправителя
        - получатель получает только amount
        - fee накапливается в self.data["bank_fee"]
        """
        sender = self.get_user(from_card)
        receiver = self.get_user(to_card)

        if not sender or not receiver:
            if sender:
                self.log_action(
                    sender,
                    "transfer",
                    "failed",
                    {"reason": "receiver_not_found", "amount": amount, "currency": CURRENCY}
                )
            return False, "Receiver card not found"

        if sender["status"] != "active":
            self.log_action(
                sender,
                "transfer",
                "failed",
                {"reason": "sender_locked", "amount": amount, "currency": CURRENCY}
            )
            return False, "Your card is locked"

        if account not in sender["accounts"]:
            self.log_action(
                sender,
                "transfer",
                "failed",
                {"reason": "no_account", "amount": amount, "currency": CURRENCY}
            )
            return False, "No such account"

        if amount <= 0:
            self.log_action(
                sender,
                "transfer",
                "failed",
                {"reason": "non_positive_amount", "amount": amount, "currency": CURRENCY}
            )
            return False, "Amount must be positive"

        # считаем комиссию: минимум 1 UZS
        fee = max(1, (amount * TRANSFER_FEE_PERCENT) // 100)
        total_debit = amount + fee

        if sender["accounts"][account] < total_debit:
            self.log_action(
                sender,
                "transfer",
                "failed",
                {
                    "reason": "insufficient_balance",
                    "amount": amount,
                    "fee": fee,
                    "total_debit": total_debit,
                    "currency": CURRENCY
                }
            )
            return False, f"Insufficient funds. Need {total_debit} {CURRENCY} including fee."

        # списываем с отправителя total_debit
        sender["accounts"][account] -= total_debit
        # зачисляем получателю только amount
        receiver["accounts"]["savings"] += amount

        # накапливаем комиссию на "счёте банка"
        self.data["bank_fee"] += fee

        now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        sender["transactions"].append({
            "time": now,
            "type": f"Transfer to {to_card}",
            "account": account,
            "amount": amount,
            "fee": fee,
            "total_debit": total_debit
        })
        receiver["transactions"].append({
            "time": now,
            "type": f"Received from {from_card}",
            "account": "savings",
            "amount": amount
        })
        self.save_data()

        # логи
        self.log_action(
            sender,
            "transfer",
            "ok",
            {
                "to": to_card,
                "account": account,
                "amount": amount,
                "fee": fee,
                "total_debit": total_debit,
                "currency": CURRENCY,
                "bank_fee_total": self.data["bank_fee"]
            }
        )
        self.log_action(
            receiver,
            "receive_transfer",
            "ok",
            {"from": from_card, "amount": amount, "currency": CURRENCY}
        )

        # возвращаем детали, чтобы интерфейс мог показать их пользователю
        info = {
            "amount": amount,
            "fee": fee,
            "total_debit": total_debit
        }
        return True, info

    def _add_tx(self, user, account, tx_type, amount, extra=None):
        tx = {
            "time": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "type": tx_type,
            "account": account,
            "amount": amount
        }
        if extra:
            tx.update(extra)
        user["transactions"].append(tx)

    def get_transactions(self, card, limit=None):
        user = self.get_user(card)
        if not user:
            return []
        txs = user["transactions"]
        if limit:
            return txs[-limit:]
        return txs

    # ---- Bank fee info ----
    def get_bank_fee_total(self):
        return self.data.get("bank_fee", 0)

    def get_bank_fee_by_date(self, date_str):
        """
        Считает сумму комиссии (fee) за конкретную дату (YYYY-MM-DD) по логам.
        Ищем action == 'transfer', status == 'ok', details['fee'].
        """
        total = 0
        for log in self.logs:
            if log.get("action") == "transfer" and log.get("status") == "ok":
                dt = log.get("datetime", "")
                if dt.startswith(date_str):
                    details = log.get("details", {})
                    fee = details.get("fee")
                    if isinstance(fee, int):
                        total += fee
        return total

    def get_bank_fee_for_period(self, start_date_str, end_date_str):
        """
        Считает сумму комиссии за период [start_date; end_date], оба включительно.
        Формат дат: YYYY-MM-DD
        """
        try:
            start_d = datetime.strptime(start_date_str, "%Y-%m-%d").date()
            end_d = datetime.strptime(end_date_str, "%Y-%m-%d").date()
        except ValueError:
            return None  # неверный формат даты

        if end_d < start_d:
            return None  # неправильный диапазон

        total = 0
        for log in self.logs:
            if log.get("action") == "transfer" and log.get("status") == "ok":
                dt_str = log.get("datetime", "")[:10]
                try:
                    d = datetime.strptime(dt_str, "%Y-%m-%d").date()
                except ValueError:
                    continue
                if start_d <= d <= end_d:
                    details = log.get("details", {})
                    fee = details.get("fee")
                    if isinstance(fee, int):
                        total += fee
        return total


# ========== ATM CLASS ==========
class ATM:
    def __init__(self, bank):
        self.bank = bank

    def user_login(self):
        card = input("Enter card number: ")
        user = self.bank.get_user(card)
        if not user:
            print("❌ Card not found")
            return None, None
        if user["status"] == "locked":
            print("❌ This card is locked.")
            return None, None

        for _ in range(3):
            pin = input("Enter PIN: ")
            ok, msg = self.bank.check_pin(card, pin)
            if ok:
                print(f"✅ Welcome, {user.get('name','user')}!")
                print(f"Your internal ID (hash): {user.get('user_id')}\n")
                return card, user
            else:
                print("⚠", msg)
                if "locked" in msg.lower():
                    return None, None
        return None, None


# ========== MAIN FUNCTION ==========
def main():
    bank = Bank()
    atm = ATM(bank)

    while True:
        print(f"""
============================
      ADVANCED ATM SYSTEM
============================
1. User Login
2. Admin Login
3. Exit
====== Currency: {CURRENCY} ======
""")
        try:
            choice = int(input("Choose: "))
        except:
            print("⚠ Invalid input! Enter a number.")
            continue

        # ---------- USER PART ----------
        if choice == 1:
            card, user = atm.user_login()
            if not user:
                continue

            # USER MENU
            while True:
                print(f"""
========= USER MENU =========
1. Check Balance
2. Deposit Money
3. Withdraw Money
4. Transfer to Another Card (1% fee)
5. View Transactions
6. Change PIN
7. Transfer Between Own Accounts
8. Logout
====== Currency: {CURRENCY} ======
""")
                try:
                    option = int(input("Choose option: "))
                except:
                    print("⚠ Enter a valid number")
                    continue

                # 1. Check Balance
                if option == 1:
                    acct = input("Account (savings/current): ")
                    bal = bank.get_balance(card, acct)
                    if bal is None:
                        print("❌ Unknown account")
                    else:
                        print(f"Balance on {acct}: {bal} {CURRENCY}")

                # 2. Deposit
                elif option == 2:
                    acct = input("Account (savings/current): ")
                    try:
                        amt = int(input(f"Amount ({CURRENCY}): "))
                    except:
                        print("⚠ Invalid amount")
                        continue
                    confirm = input(
                        f"Confirm deposit {amt} {CURRENCY} to {acct}? (y/n): "
                    ).lower()
                    if confirm != "y":
                        print("❌ Deposit cancelled")
                        bank.log_action(
                            user,
                            "deposit",
                            "cancelled",
                            {"account": acct, "amount": amt, "currency": CURRENCY}
                        )
                        continue
                    if bank.deposit(card, acct, amt):
                        print(f"✅ Deposit successful! ({amt} {CURRENCY} to {acct})")
                        send_sms_mock(user.get("phone"), f"Deposit {amt} {CURRENCY} to {acct} completed.")
                    else:
                        print("❌ Deposit failed")

                # 3. Withdraw
                elif option == 3:
                    acct = input("Account (savings/current): ")
                    try:
                        amt = int(input(f"Amount ({CURRENCY}): "))
                    except:
                        print("⚠ Invalid amount")
                        continue
                    confirm = input(
                        f"Confirm withdraw {amt} {CURRENCY} from {acct}? (y/n): "
                    ).lower()
                    if confirm != "y":
                        print("❌ Withdrawal cancelled")
                        bank.log_action(
                            user,
                            "withdraw",
                            "cancelled",
                            {"account": acct, "amount": amt, "currency": CURRENCY}
                        )
                        continue
                    ok, msg = bank.withdraw(card, acct, amt)
                    if ok:
                        print(f"✅ Withdraw Successful! {amt} {CURRENCY}. Notes: {msg}")
                        send_sms_mock(user.get("phone"), f"Withdrawal {amt} {CURRENCY} from {acct} completed.")
                    else:
                        print("❌", msg)

                # 4. Transfer to Another Card (1% fee)
                elif option == 4:
                    to_card = input("Receiver card number: ")
                    acct = input("Your account (savings/current): ")
                    try:
                        amt = int(input(f"Amount to transfer ({CURRENCY}): "))
                    except:
                        print("⚠ Invalid amount")
                        continue
                    confirm = input(
                        f"Confirm transfer {amt} {CURRENCY} from {acct} to card {to_card} (1% fee)? (y/n): "
                    ).lower()
                    if confirm != "y":
                        print("❌ Transfer cancelled")
                        bank.log_action(
                            user,
                            "transfer",
                            "cancelled",
                            {"to": to_card, "account": acct, "amount": amt, "currency": CURRENCY}
                        )
                        continue

                    ok, info_or_msg = bank.transfer(card, to_card, acct, amt)
                    if ok:
                        info = info_or_msg
                        amount = info["amount"]
                        fee = info["fee"]
                        total_debit = info["total_debit"]
                        print(
                            f"✅ Transfer successful!\n"
                            f"   Sent: {amount} {CURRENCY}\n"
                            f"   Fee: {fee} {CURRENCY}\n"
                            f"   Total debited: {total_debit} {CURRENCY}"
                        )
                        send_sms_mock(
                            user.get("phone"),
                            f"Transfer {amount} {CURRENCY} to card {to_card} completed. "
                            f"Fee: {fee} {CURRENCY}. Total debited: {total_debit} {CURRENCY}."
                        )
                    else:
                        print("❌", info_or_msg)

                # 5. View Transactions
                elif option == 5:
                    txs = bank.get_transactions(card, limit=10)
                    print("\n--- LAST TRANSACTIONS ---")
                    if not txs:
                        print("No transactions yet.")
                    else:
                        for t in txs:
                            base = f"{t['time']} | {t['type']} | {t['account']} | {t['amount']} {CURRENCY}"
                            fee_part = ""
                            if "fee" in t:
                                fee_part = f" | fee: {t['fee']} {CURRENCY}"
                            if "total_debit" in t:
                                fee_part += f" | total: {t['total_debit']} {CURRENCY}"
                            print(base + fee_part)
                    print("-------------------------\n")

                # 6. Change PIN
                elif option == 6:
                    old_pin = input("Enter current PIN: ")
                    ok, msg = bank.check_pin(card, old_pin)
                    if not ok:
                        print("❌", msg)
                        continue
                    new_pin = input("New PIN: ")
                    confirm_pin = input("Confirm new PIN: ")
                    if new_pin != confirm_pin:
                        print("❌ PINs do not match")
                        bank.log_action(user, "change_pin", "failed", {"reason": "mismatch"})
                        continue
                    user["pin"] = str(new_pin)
                    bank.save_data()
                    bank.log_action(user, "change_pin", "ok", {})
                    print("✅ PIN updated")
                    send_sms_mock(user.get("phone"), "Your PIN has been changed.")

                # 7. Transfer Between Own Accounts
                elif option == 7:
                    from_acc = input("From (savings/current): ")
                    to_acc = input("To (savings/current): ")
                    try:
                        amt = int(input(f"Amount ({CURRENCY}): "))
                    except:
                        print("⚠ Invalid amount")
                        continue
                    confirm = input(
                        f"Confirm transfer {amt} {CURRENCY} from {from_acc} to {to_acc}? (y/n): "
                    ).lower()
                    if confirm != "y":
                        print("❌ Internal transfer cancelled")
                        bank.log_action(
                            user,
                            "internal_transfer",
                            "cancelled",
                            {"from": from_acc, "to": to_acc, "amount": amt, "currency": CURRENCY}
                        )
                        continue
                    u = bank.get_user(card)
                    if (from_acc not in u["accounts"] or
                        to_acc not in u["accounts"] or
                        u["accounts"][from_acc] < amt or amt <= 0):
                        print("❌ Cannot transfer")
                        bank.log_action(
                            user,
                            "internal_transfer",
                            "failed",
                            {"from": from_acc, "to": to_acc, "amount": amt, "currency": CURRENCY}
                        )
                    else:
                        u["accounts"][from_acc] -= amt
                        u["accounts"][to_acc] += amt
                        bank.save_data()
                        bank.log_action(
                            user,
                            "internal_transfer",
                            "ok",
                            {"from": from_acc, "to": to_acc, "amount": amt, "currency": CURRENCY}
                        )
                        print(f"✅ Transfer between accounts done ({amt} {CURRENCY})")
                        send_sms_mock(
                            user.get("phone"),
                            f"Internal transfer {amt} {CURRENCY} from {from_acc} to {to_acc} completed."
                        )

                # 8. Logout
                elif option == 8:
                    break

                else:
                    print("Invalid option!")

        # ---------- ADMIN PART ----------
        elif choice == 2:
            username = input("Username: ")
            password = input("Password: ")
            if not bank.validate_admin(username, password):
                print("Wrong credentials")
                continue
            print("Admin logged in!")

            while True:
                print(f"""
===== ADMIN PANEL =====
1. Add User
2. Refill ATM Cash
3. Check ATM Cash
4. View Security Logs by card number
5. View Security Logs by user_id
6. View Bank Fee Account (total)
7. View Bank Fee Report by Date
8. View Bank Fee Report for Period (custom dates)
9. Quick Preset Fee Reports (today / 7 days / 30 days)
10. Exit Admin Panel
====== Currency: {CURRENCY} ======
""")
                try:
                    admin_option = int(input("Choose: "))
                except:
                    print("Invalid")
                    continue

                # 1. Add User
                if admin_option == 1:
                    card = input("New user card number: ")
                    pin = input("Set PIN: ")
                    name = input("User name: ")
                    phone = input("User phone (for SMS): ")
                    ok, msg = bank.add_user(card, pin, name, phone)
                    print("✅" if ok else "❌", msg)

                # 2. Refill ATM cash
                elif admin_option == 2:
                    print("Current ATM cash:", bank.data["atm_cash"])
                    for note in ["2000", "500", "200", "100"]:
                        try:
                            add = int(input(f"Add how many {note} notes? "))
                        except:
                            add = 0
                        bank.data["atm_cash"][note] += add
                    bank.save_data()
                    print("✅ ATM cash updated")

                # 3. Check ATM cash
                elif admin_option == 3:
                    total = 0
                    print("\n--- ATM CASH ---")
                    for note, cnt in bank.data["atm_cash"].items():
                        value = int(note) * cnt
                        total += value
                        print(f"{note}: {cnt} notes (value {value} {CURRENCY})")
                    print(f"TOTAL: {total} {CURRENCY}")
                    print("----------------\n")

                # 4. View logs by card number
                elif admin_option == 4:
                    card = input("Enter card number: ").strip()
                    try:
                        limit = int(input("How many last records to show? (0 = all): "))
                    except:
                        limit = 10
                    if limit <= 0:
                        logs = bank.get_logs_by_card(card, None)
                    else:
                        logs = bank.get_logs_by_card(card, limit)
                    print(f"\n--- SECURITY LOGS for card={card} ---")
                    if not logs:
                        print("No logs for this card.")
                    else:
                        for log in logs:
                            print(
                                f"{log['datetime']} | {log['action']} | "
                                f"status={log['status']} | details={log.get('details', {})}"
                            )
                    print("--------------------------------------\n")

                # 5. View logs by user_id
                elif admin_option == 5:
                    user_id = input("Enter user_id (hash): ").strip()
                    try:
                        limit = int(input("How many last records to show? (0 = all): "))
                    except:
                        limit = 10
                    if limit <= 0:
                        logs = bank.get_logs_by_user_id(user_id, None)
                    else:
                        logs = bank.get_logs_by_user_id(user_id, limit)
                    print(f"\n--- SECURITY LOGS for user_id={user_id} ---")
                    if not logs:
                        print("No logs for this user_id.")
                    else:
                        for log in logs:
                            print(
                                f"{log['datetime']} | {log['action']} | "
                                f"status={log['status']} | details={log.get('details', {})}"
                            )
                    print("-----------------------------------------\n")

                # 6. View Bank Fee Account (total)
                elif admin_option == 6:
                    total_fee = bank.get_bank_fee_total()
                    print(f"\n💰 Total collected transfer fees (all time): {total_fee} {CURRENCY}\n")

                # 7. View Bank Fee Report by Date
                elif admin_option == 7:
                    date_str = input("Enter date (YYYY-MM-DD): ").strip()
                    if not date_str:
                        print("❌ Date is empty")
                        continue
                    fee_for_date = bank.get_bank_fee_by_date(date_str)
                    print(f"\n📆 Fee collected on {date_str}: {fee_for_date} {CURRENCY}\n")

                # 8. View Bank Fee Report for Period (custom)
                elif admin_option == 8:
                    start_date = input("Start date (YYYY-MM-DD): ").strip()
                    end_date = input("End date   (YYYY-MM-DD): ").strip()
                    if not start_date or not end_date:
                        print("❌ Dates cannot be empty")
                        continue
                    total_period = bank.get_bank_fee_for_period(start_date, end_date)
                    if total_period is None:
                        print("❌ Invalid date format or range")
                    else:
                        print(f"\n📊 Fee collected from {start_date} to {end_date}: {total_period} {CURRENCY}\n")

                # 9. Quick Preset Fee Reports
                elif admin_option == 9:
                    print("""
--- QUICK FEE REPORTS ---
1. Today
2. Last 7 days
3. Last 30 days
""")
                    try:
                        preset = int(input("Choose preset: "))
                    except:
                        print("Invalid preset")
                        continue

                    today = date.today()

                    if preset == 1:
                        start_d = today
                        end_d = today
                        label = "today"
                    elif preset == 2:
                        start_d = today - timedelta(days=6)
                        end_d = today
                        label = "last 7 days"
                    elif preset == 3:
                        start_d = today - timedelta(days=29)
                        end_d = today
                        label = "last 30 days"
                    else:
                        print("Invalid preset")
                        continue

                    start_str = start_d.strftime("%Y-%m-%d")
                    end_str = end_d.strftime("%Y-%m-%d")
                    total_period = bank.get_bank_fee_for_period(start_str, end_str)
                    print(f"\n📊 Fee collected for {label} ({start_str} .. {end_str}): {total_period} {CURRENCY}\n")

                # 10. Exit admin panel
                elif admin_option == 10:
                    break

                else:
                    print("Invalid admin option!")

        # ---------- EXIT ----------
        elif choice == 3:
            print("Goodbye!")
            break

        else:
            print("Invalid!")


# ========== START PROGRAM ==========
if __name__ == "__main__":
    main()



      ADVANCED ATM SYSTEM
1. User Login
2. Admin Login
3. Exit
====== Currency: UZS ======

Choose: 1
Enter card number: admin
❌ Card not found

      ADVANCED ATM SYSTEM
1. User Login
2. Admin Login
3. Exit
====== Currency: UZS ======

Choose: 1
Enter card number: 1111
❌ Card not found

      ADVANCED ATM SYSTEM
1. User Login
2. Admin Login
3. Exit
====== Currency: UZS ======

Choose: 2
Username: admin123
Password: admin123
Wrong credentials

      ADVANCED ATM SYSTEM
1. User Login
2. Admin Login
3. Exit
====== Currency: UZS ======

Choose: 2
Username: admin
Password: admin123
Admin logged in!

===== ADMIN PANEL =====
1. Add User
2. Refill ATM Cash
3. Check ATM Cash
4. View Security Logs by card number
5. View Security Logs by user_id
6. View Bank Fee Account (total)
7. View Bank Fee Report by Date
8. View Bank Fee Report for Period (custom dates)
9. Quick Preset Fee Reports (today / 7 days / 30 days)
10. Exit Admin Panel
====== Currency: UZS ======

Choose: 1
New user card number: 1